In [1]:
"""
===============================================================
Custom Dictionary Implementation in Python (Haris_dict)
===============================================================
Author: Muhammad Haris

Description:
    This module implements a custom dictionary (hash map) from 
    scratch using **open addressing with linear probing** and 
    **tombstones** to handle deletions.

    Features supported:
        - Insert (with update support for existing keys)
        - Delete (with tombstone markers for proper probing)
        - Get/Search by key
        - Automatic resize when load factor exceeds 0.7
        - Iteration over keys
        - Utility methods: items(), keys_list(), values_list(), clear()
        - Python magic methods for len(), contains, get/set/delete

    Internals:
        - Uses two parallel arrays: `keys` and `values`
        - Uses `Tombstone` markers to allow deletes without breaking probe chains
        - Resizes (rehashes) automatically to maintain efficiency
        - Average O(1) for insert, get, delete; worst-case O(n)

Learning Note:
    This project was first practiced in raw form to understand 
    hash maps and probing. With GPT's assistance, it was later 
    refined for:
        • clear method coverage and resizing logic
        • structured headings and docstrings
        • systematic, step-by-step testing

Purpose:
    Built for educational purposes to deeply understand how 
    Python dictionaries work under the hood, and to strengthen 
    problem-solving skills in data structures.
===============================================================
"""

# ===============================
# Special marker for deleted slots
# ===============================
Tombstone = object()

# ===============================
# Dictionary Class
# ===============================
class Dictionary:
    """
    A custom hash map implementation using open addressing 
    with linear probing and tombstones.

    Attributes:
        size   (int): total capacity of the table
        count  (int): number of active key-value pairs
        keys   (list): array storing keys
        values (list): array storing corresponding values

    Main methods:
        put(key, value)     -> insert or update a key-value pair
        get(key)            -> retrieve value by key
        delete(key)         -> remove a key (mark with Tombstone)
        __resize__(new_cap) -> resize table when load factor > 0.7
        clear()             -> reset dictionary
        items()             -> return list of (key, value) pairs
        keys_list()         -> return list of keys
        values_list()       -> return list of values
    """

    # ---------------------------
    # Constructor
    # ---------------------------
    def __init__(self, size=5):
        self.size = size
        self.keys = [None] * self.size
        self.values = [None] * self.size
        self.count = 0

    # ---------------------------
    # Core Put Method (Insert/Update)
    # ---------------------------
    def put(self, key, value):
        """Insert or update key-value pair, resizing if needed."""
        if self.count >= int(self.size * 0.7):
            self.__resize__(self.size * 2)

        index = abs(hash(key)) % self.size
        start_index = index
        first_tomb = None

        while self.keys[index] is not None:
            if self.keys[index] == key:       # update existing key
                self.values[index] = value
                return
            if self.keys[index] is Tombstone and first_tomb is None:
                first_tomb = index            # remember first tombstone
            index = (index + 1) % self.size
            if index == start_index:
                raise Exception("Dictionary is full, cannot insert")

        # If tombstone found earlier, reuse it
        if first_tomb is not None:
            index = first_tomb
        else:
            self.count += 1   # count only fresh slots

        self.keys[index] = key
        self.values[index] = value

    # ---------------------------
    # Delete Method
    # ---------------------------
    def delete(self, key):
        """Delete a key (replace with Tombstone marker)."""
        index = abs(hash(key)) % self.size
        start_index = index
        while self.keys[index] is not None:
            if self.keys[index] == key:
                self.keys[index] = Tombstone
                self.values[index] = None
                self.count -= 1
                return
            index = (index + 1) % self.size
            if index == start_index:
                break
        raise KeyError(f"Key {key} not found")

    # ---------------------------
    # Get Method
    # ---------------------------
    def get(self, key):
        """Retrieve value for given key (raises KeyError if missing)."""
        index = abs(hash(key)) % self.size
        start_index = index
        while self.keys[index] is not None:
            if self.keys[index] == key:
                return self.values[index]
            index = (index + 1) % self.size
            if index == start_index:
                break
        raise KeyError(f"Key {key} not found")

    # ---------------------------
    # Built-in Magic Methods
    # ---------------------------
    def __len__(self):
        return self.count

    def __contains__(self, key):
        try:
            self.get(key)
            return True
        except KeyError:
            return False

    def __getitem__(self, key):
        return self.get(key)

    def __setitem__(self, key, value):
        self.put(key, value)

    def __delitem__(self, key):
        self.delete(key)

    def __iter__(self):
        for k in self.keys:
            if k not in (None, Tombstone):
                yield k

    # ---------------------------
    # Resize Method
    # ---------------------------
    def __resize__(self, new_capacity):
        """Resize the dictionary and rehash existing items."""
        old_keys = self.keys
        old_values = self.values
        self.size = new_capacity
        self.keys = [None] * self.size
        self.values = [None] * self.size
        self.count = 0
        for k, v in zip(old_keys, old_values):
            if k not in (None, Tombstone):
                self.put(k, v)

    # ---------------------------
    # Utility Methods
    # ---------------------------
    def clear(self):
        """Remove all items from dictionary."""
        self.keys = [None] * self.size
        self.values = [None] * self.size
        self.count = 0

    def items(self):
        """Return all (key, value) pairs."""
        return [(k, v) for k, v in zip(self.keys, self.values) if k not in (None, Tombstone)]

    def keys_list(self):
        """Return all keys."""
        return [k for k in self.keys if k not in (None, Tombstone)]

    def values_list(self):
        """Return all values."""
        return [v for k, v in zip(self.keys, self.values) if k not in (None, Tombstone)]

    def __str__(self):
        """Readable string showing dictionary details."""
        pairs = ", ".join(f"{k}: {v}" for k, v in self.items())
        load_factor = round(self.count / self.size, 2) if self.size else 0
        return (
            f"Dictionary(\n"
            f"  size = {self.size}\n"
            f"  count = {self.count}\n"
            f"  load_factor = {load_factor}\n"
            f"  data = {{ {pairs} }}\n"
            f")"
        )

# ===============================
# Testing the Dictionary
# ===============================
"""
===============================================================
Testing Section for Haris_dict
===============================================================
This section demonstrates step-by-step usage of the Dictionary:

    1. Insert multiple items
    2. Access values, keys, items
    3. Update values for existing keys
    4. Delete keys (tombstone behavior)
    5. Iterate through dictionary
    6. Automatic resizing when load factor > 0.7
    7. Clear entire dictionary

Purpose:
    Ensures all methods work as intended, and provides
    clear examples for learning and debugging.
===============================================================
"""

if __name__ == "__main__":
    d = Dictionary()

    print("=== INSERT ===")
    d[1] = "one"
    d[2] = "two"
    d[3] = "three"
    print(d)

    print("\n=== ACCESS ===")
    print("Key 2 ->", d[2])
    print("Keys:", d.keys_list())
    print("Values:", d.values_list())
    print("Items:", d.items())

    print("\n=== UPDATE ===")
    d[2] = "TWO"
    print("Updated key 2 ->", d[2])
    print(d)

    print("\n=== DELETE ===")
    del d[3]
    print("After deleting key 3:")
    print(d)

    print("\n=== ITERATION ===")
    for k in d:
        print(k, "->", d[k])

    print("\n=== RESIZE DEMO ===")
    for i in range(4, 15):   # Force resize
        d[i] = str(i)
    print(d)

    print("\n=== CLEAR ===")
    d.clear()
    print("After clear:", d)

=== INSERT ===
Dictionary(
  size = 5
  count = 3
  load_factor = 0.6
  data = { 1: one, 2: two, 3: three }
)

=== ACCESS ===
Key 2 -> two
Keys: [1, 2, 3]
Values: ['one', 'two', 'three']
Items: [(1, 'one'), (2, 'two'), (3, 'three')]

=== UPDATE ===
Updated key 2 -> TWO
Dictionary(
  size = 10
  count = 3
  load_factor = 0.3
  data = { 1: one, 2: TWO, 3: three }
)

=== DELETE ===
After deleting key 3:
Dictionary(
  size = 10
  count = 2
  load_factor = 0.2
  data = { 1: one, 2: TWO }
)

=== ITERATION ===
1 -> one
2 -> TWO

=== RESIZE DEMO ===
Dictionary(
  size = 20
  count = 13
  load_factor = 0.65
  data = { 1: one, 2: TWO, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14 }
)

=== CLEAR ===
After clear: Dictionary(
  size = 20
  count = 0
  load_factor = 0.0
  data = {  }
)
